In [ ]:
import avstack
import avapi
import os
import shutil
from tqdm import tqdm

In [ ]:
train_folder = '/data/spencer/KITTI/object/train_only'
val_folder = '/data/spencer/KITTI/object/val_only'
obj_data_dir = '/data/spencer/KITTI/object'
folders = [train_folder, val_folder]
subfolders = ['velodyne', 'image_2', 'image_3', 'planes', 'label_2', 'calib']
exts       = ['.bin',     '.png',    '.png',    '.txt',   '.txt',    '.txt']
for fol in folders:
    os.makedirs(fol, exist_ok=True)
    os.makedirs(fol, exist_ok=True)
    for sub in subfolders:
        os.makedirs(os.path.join(fol, sub), exist_ok=True)

In [ ]:
KDM = avapi.kitti.KittiObjectDataset(obj_data_dir, 'training')
with open(os.path.join('/data/spencer/KITTI/object/ImageSets/train.txt'), 'r') as f:
    idxs_train_only = [int(s.strip()) for s in f.readlines()]
with open(os.path.join('/data/spencer/KITTI/object/ImageSets/val.txt'), 'r') as f:
    idxs_val_only = [int(s.strip()) for s in f.readlines()]
idxs = [idxs_train_only, idxs_val_only]

for idxs_this, fol in zip(idxs, folders):
    # Write image sets
    fol_split = fol.split('/')
    write_str = '\n'.join(['%06d'%i for i in idxs_this])
    with open('/' + os.path.join(*fol_split[:-1], 'ImageSets', fol_split[-1] + '.txt'), 'w') as f:
        f.write(write_str)
              
    # Copy data
    for ext, sub in zip(exts, subfolders):
        print(f'Copying {os.path.join(fol, sub)}')
        for idx in tqdm(idxs_this):
            src = os.path.join(KDM.split_path, sub, '%06d'%idx + ext)
            dest = os.path.join(fol, sub, '%06d'%idx + ext)
            shutil.copy2(src, dest)